# Prepare Sig Generation

In [41]:
from subprocess import call
import os
from tqdm import tqdm
import pandas as pd
from glob import glob
from pathlib import Path

## load all cve and commit info

In [198]:
cve_and_bin_func = pd.read_csv("./software_dataset/cve_all", header=None, names=['CVE', "vul_bin", "patch_bin", "func"])
cve_commits = pd.read_csv("./software_dataset/commits_all_cve.txt", header=None, names=['CVE', 'commits'])
cve_and_bin_func = cve_and_bin_func.set_index("CVE")

In [ ]:
cve_commit = cve_commits.dropna()
keep_one_commit = lambda x: x.strip().split(" ")[0]
cve_commit['commits'] = cve_commit['commits'].apply(keep_one_commit)

In [200]:
cve_commit.head()

,CVE,commits
0,CVE-2010-2805,45a3c76b547511fa9d97aca34b150a0663257375
1,CVE-2010-2806,c06da1ad34663da7b6fc39b030dc3ae185b96557
2,CVE-2010-2807,346f1867fd32dae8f56e5b482d1af98f626804ac
3,CVE-2010-2808,81f3472c0ba7b8f6466e2e214fa8c1c17fade975
4,CVE-2010-3814,0edf0986f3be570f5bf90ff245a85c1675f5c9a4


In [19]:
cve_and_bin_func.head()

,vul_bin,patch_bin,func
CVE,,,
CVE-2010-2805,binaries/freetype/O0/freetype-2.4.1,binaries/freetype/O0/freetype-2.4.2,FT_Stream_EnterFrame
CVE-2010-2806,binaries/freetype/O0/freetype-2.4.1,binaries/freetype/O0/freetype-2.4.2,t42_parse_sfnts
CVE-2010-2807,binaries/freetype/O0/freetype-2.4.1,binaries/freetype/O0/freetype-2.4.2,BOUNDSL
CVE-2010-2808,binaries/freetype/O0/freetype-2.4.1,binaries/freetype/O0/freetype-2.4.2,Mac_Read_POST_Resource
CVE-2010-3814,binaries/freetype/O0/freetype-2.4.3,binaries/freetype/O0/freetype-2.4.4,Ins_SHZ


In [201]:
df_cve_info = pd.merge(cve_and_bin_func, cve_commit, on = "CVE", how="inner")

In [202]:
df_cve_info = df_cve_info.dropna()

In [203]:
df_cve_info['CVE'].nunique()

210

## Generate patch & symbol file

In [128]:
PATCH_SAVE_DIR = "/home/ubuntu/fiber/software_dataset/patches"

In [156]:
def convert_bin_path(bin_path):
    sp = bin_path.split('/')
    bin_name= sp[-2:]
    software_name = sp[1]
    bin_file = os.path.join("/extended/", software_name, "binary", *bin_name)
    
    if "binutils" in bin_file:
        to_do_path = Path(bin_file)
        binary_name = to_do_path.name
        name, version = binary_name.split("-")
        bin_file = to_do_path.parent/ Path("binutils-{}-{}".format(version, name))
    
    return str(bin_file)

In [157]:
def generate_sym(bin_path, save_path):
    if os.path.exists(save_path):
        return
    cmd = "nm " + bin_path + " | egrep '^[0-9]' " + " > " + save_path
    call(cmd, shell=True)

In [158]:
patch_file_list = []
for idx, (cveid, _, patch_bin, _, commit_id) in df_cve_info.iterrows():
    patch_bin_file = convert_bin_path(patch_bin)
    software_name = patch_bin_file.split("/")[2]
    code_repo_dir = Path(patch_bin_file).parent.parent.parent / Path("repo") / Path(software_name)
    # patch generation by using `git show commidId`
    patch_file = os.path.join(PATCH_SAVE_DIR, cveid)
    patch_info_file = "software_dataset/patches/patch_info_"+cveid
    if not os.path.exists(patch_file):
        cmd = "cd " + str(code_repo_dir) + "&&" + "git show "+commit_id + " > " + patch_file
        print(cmd)
        ret = call(cmd, shell=True)
        print(ret)
        
        with open(patch_info_file, "w") as f:
            f.write(patch_file)
    
    software_name = patch_bin.split('/')[-1]
    # generate symbol file
    sym_file = patch_bin_file+".sym"
    generate_sym(patch_bin_file, sym_file)
    patch_file_list.append([cveid, patch_info_file, str(Path(patch_bin_file).parent.parent.parent/Path("source_code")/Path(software_name)), sym_file])

In [159]:
len(patch_file_list),patch_file_list

(533,
 [['CVE-2010-2805',
   'software_dataset/patches/patch_info_CVE-2010-2805',
   '/extended/freetype/source_code/freetype-2.4.2',
   '/extended/freetype/binary/O0/freetype-2.4.2.sym'],
  ['CVE-2010-2806',
   'software_dataset/patches/patch_info_CVE-2010-2806',
   '/extended/freetype/source_code/freetype-2.4.2',
   '/extended/freetype/binary/O0/freetype-2.4.2.sym'],
  ['CVE-2010-2807',
   'software_dataset/patches/patch_info_CVE-2010-2807',
   '/extended/freetype/source_code/freetype-2.4.2',
   '/extended/freetype/binary/O0/freetype-2.4.2.sym'],
  ['CVE-2010-2808',
   'software_dataset/patches/patch_info_CVE-2010-2808',
   '/extended/freetype/source_code/freetype-2.4.2',
   '/extended/freetype/binary/O0/freetype-2.4.2.sym'],
  ['CVE-2010-3814',
   'software_dataset/patches/patch_info_CVE-2010-3814',
   '/extended/freetype/source_code/freetype-2.4.4',
   '/extended/freetype/binary/O0/freetype-2.4.4.sym'],
  ['CVE-2010-3855',
   'software_dataset/patches/patch_info_CVE-2010-3855',
   

# Picker:  signature generation

In [160]:
OUTPUT_DIR = "/home/ubuntu/fiber/software_dataset/outputs"

In [161]:
ext_files=[]
for c in patch_file_list:
    output_file = os.path.join(OUTPUT_DIR, c[0]+".output")
    cmd = "python pick_sig.py " + c[1] + " " + c[2] + " " + output_file + " " + c[3]
    print(cmd)
    print(call(cmd, shell=True))
    ext_files.append(output_file)

python pick_sig.py software_dataset/patches/patch_info_CVE-2010-2805 /extended/freetype/source_code/freetype-2.4.2 /home/ubuntu/fiber/software_dataset/outputs/CVE-2010-2805.output /extended/freetype/binary/O0/freetype-2.4.2.sym
0
python pick_sig.py software_dataset/patches/patch_info_CVE-2010-2806 /extended/freetype/source_code/freetype-2.4.2 /home/ubuntu/fiber/software_dataset/outputs/CVE-2010-2806.output /extended/freetype/binary/O0/freetype-2.4.2.sym
0
python pick_sig.py software_dataset/patches/patch_info_CVE-2010-2807 /extended/freetype/source_code/freetype-2.4.2 /home/ubuntu/fiber/software_dataset/outputs/CVE-2010-2807.output /extended/freetype/binary/O0/freetype-2.4.2.sym
0
python pick_sig.py software_dataset/patches/patch_info_CVE-2010-2808 /extended/freetype/source_code/freetype-2.4.2 /home/ubuntu/fiber/software_dataset/outputs/CVE-2010-2808.output /extended/freetype/binary/O0/freetype-2.4.2.sym
0
python pick_sig.py software_dataset/patches/patch_info_CVE-2010-3814 /extended/f

In [162]:
valid_output = 0
for x in ext_files:
    if not os.path.exists(x):
        continue
    with open(x) as f:
        lines = f.readlines()
        if len(lines) > 1:
#             print(lines)
            valid_output += 1
print("valid signature file number ", valid_output)

('valid signature file number ', 161)


# Translator

In [164]:
for c, ext_file in zip(patch_file_list,ext_files):
    bin_path = c[3][:-4]  
    cmd = "python ext_sig.py {} {} {} {} {} ".format(bin_path, c[3], bin_path, ext_file, OUTPUT_DIR)
    print(cmd)
    print(call(cmd, shell=True))

python ext_sig.py /extended/freetype/binary/O0/freetype-2.4.2 /extended/freetype/binary/O0/freetype-2.4.2.sym /extended/freetype/binary/O0/freetype-2.4.2 /home/ubuntu/fiber/software_dataset/outputs/CVE-2010-2805.output /home/ubuntu/fiber/software_dataset/outputs 
0
python ext_sig.py /extended/freetype/binary/O0/freetype-2.4.2 /extended/freetype/binary/O0/freetype-2.4.2.sym /extended/freetype/binary/O0/freetype-2.4.2 /home/ubuntu/fiber/software_dataset/outputs/CVE-2010-2806.output /home/ubuntu/fiber/software_dataset/outputs 
0
python ext_sig.py /extended/freetype/binary/O0/freetype-2.4.2 /extended/freetype/binary/O0/freetype-2.4.2.sym /extended/freetype/binary/O0/freetype-2.4.2 /home/ubuntu/fiber/software_dataset/outputs/CVE-2010-2807.output /home/ubuntu/fiber/software_dataset/outputs 
0
python ext_sig.py /extended/freetype/binary/O0/freetype-2.4.2 /extended/freetype/binary/O0/freetype-2.4.2.sym /extended/freetype/binary/O0/freetype-2.4.2 /home/ubuntu/fiber/software_dataset/outputs/CVE-

# Matcher

In [168]:
from glob import glob
from collections import defaultdict

## load all generated signature files

In [169]:
signature_file = OUTPUT_DIR + "/*-sig*"
sig_file_dic = defaultdict(list)
for sig_file in glob(signature_file):
    cve = sig_file.split("/")[-1][:-6]
    sig_file_dic[cve].append(sig_file)

In [170]:
len(sig_file_dic)

65

* supported CVEs

In [193]:
sig_file_dic.keys()

['CVE-2015-0293',
 'CVE-2017-13010',
 'CVE-2016-7122',
 'CVE-2016-10190',
 'CVE-2016-10191',
 'CVE-2015-1793',
 'CVE-2018-7751',
 'CVE-2017-13037',
 'CVE-2016-7905',
 'CVE-2014-8275',
 'CVE-2017-5025',
 'CVE-2014-8176',
 'CVE-2015-1788',
 'CVE-2014-0221',
 'CVE-2012-2686',
 'CVE-2017-12993',
 'CVE-2017-12897',
 'CVE-2017-12894',
 'CVE-2017-12990',
 'CVE-2013-6450',
 'CVE-2014-3568',
 'CVE-2014-3569',
 'CVE-2014-3508',
 'CVE-2017-11665',
 'CVE-2015-0288',
 'CVE-2017-9995',
 'CVE-2017-13044',
 'CVE-2017-14058',
 'CVE-2017-14767',
 'CVE-2016-6304',
 'CVE-2017-14054',
 'CVE-2017-14055',
 'CVE-2017-14056',
 'CVE-2017-14059',
 'CVE-2017-9993',
 'CVE-2015-1790',
 'CVE-2017-16840',
 'CVE-2018-6392',
 'CVE-2016-0702',
 'CVE-2016-0705',
 'CVE-2016-2180',
 'CVE-2015-1792',
 'CVE-2015-3197',
 'CVE-2017-12896',
 'CVE-2017-14171',
 'CVE-2015-0206',
 'CVE-2017-13018',
 'CVE-2017-13690',
 'CVE-2016-2109',
 'CVE-2014-0224',
 'CVE-2016-7502',
 'CVE-2014-3512',
 'CVE-2017-13013',
 'CVE-2017-12899',
 'CVE

## Load target info

In [171]:
import pandas as pd

In [249]:
target_file = "./software_dataset/detection_optim_O0"
targets = pd.read_csv(target_file, header=None, names=["CVE", "software",'func','label'])
targets = targets.set_index("CVE")
targets_all = targets.copy()

In [250]:
# targets = targets[targets["software"].str.contains("openssl")]

In [251]:
targets.head()

,software,func,label
CVE,,,
CVE-2014-0160,binaries/openssl/O0/openssl-1.0.1a,dtls1_process_heartbeat,-1
CVE-2014-0160,binaries/openssl/O0/openssl-1.0.1e,dtls1_process_heartbeat,-1
CVE-2014-0160,binaries/openssl/O0/openssl-1.0.1h,dtls1_process_heartbeat,1
CVE-2014-0160,binaries/openssl/O0/openssl-1.0.1u,dtls1_process_heartbeat,1
CVE-2015-0288,binaries/openssl/O0/openssl-1.0.1n,X509_to_X509_REQ,1


## Detection for O0

In [174]:
for cve, sig_files in sig_file_dic.items():
    print(cve)
    if cve not in targets.index:
        continue
    target_bins = targets.loc[cve]['software'].to_list()
    
    # write sig file paths to file
    sig_list_file = os.path.join(OUTPUT_DIR,  cve+"_sig_list")
    with open(sig_list_file, "w") as f:
        for sig_path in sig_files:
            f.write(sig_path + " 1\n")
    
    for target_bin in target_bins:
        target_bin = convert_bin_path(target_bin)
        sym_file = target_bin+".sym"
        generate_sym(target_bin, sym_file)
        cmd = "python match_sig.py {} {} {}".format(target_bin, sym_file, sig_list_file)
        print(cmd)
        call(cmd, shell=True)
        
        # read the detection result

CVE-2015-0293
python match_sig.py /extended/openssl/binary/O0/openssl-1.0.1n /extended/openssl/binary/O0/openssl-1.0.1n.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
python match_sig.py /extended/openssl/binary/O0/openssl-1.0.1l /extended/openssl/binary/O0/openssl-1.0.1l.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
python match_sig.py /extended/openssl/binary/O0/openssl-1.0.1a /extended/openssl/binary/O0/openssl-1.0.1a.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
CVE-2017-13010
python match_sig.py /extended/tcpdump/binary/O0/tcpdump-4.9.0 /extended/tcpdump/binary/O0/tcpdump-4.9.0.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2017-13010_sig_list
python match_sig.py /extended/tcpdump/binary/O0/tcpdump-4.9.3 /extended/tcpdump/binary/O0/tcpdump-4.9.3.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2017-13010_sig_list
CVE-2016-7122
python match_sig.py /extended/ffmpeg/binary/O0/ffmpeg-3.1.2 /extended/ffmp

### Result Statistics

In [175]:
res_files = glob("match_res_*")
CVE_res = [] # (CVE, bin, P/N,Score)
Detection_times = []
for res_file in res_files:
    with open(res_file, "r") as f:
        lines = f.readlines()
        bin_name = res_file.split("_")[2]
        if len(lines)==1:
            continue
        CVE, IS_P, score = lines[0].strip().split(" ")
        CVE_res.append((CVE, bin_name, IS_P, score))

In [176]:
df_cve_res = pd.DataFrame(CVE_res, columns=["CVE", "software", "patched", "score"])

In [177]:
extract_software = lambda x: x.split("/")[-1] 
targets_all['software'] = targets_all['software'].apply(extract_software)

In [178]:
all_results = pd.merge(df_cve_res, targets_all, on=["CVE","software"], how="inner")

In [190]:
all_results.shape

(186, 6)

### Accuracy in Support

In [187]:
TPs = all_results.query("patched=='P' and label==1")
TNs = all_results.query("patched=='N' and label==-1")

In [207]:
correct_cases = TNs.shape[0] + TPs.shape[0]
accuracy = correct_cases*1.0/all_results.shape[0]
correct_cases,accuracy

(102, 0.5483870967741935)

## Detection for O1

In [276]:
# delete all match result files.
for filename in Path("./").glob("match_res_*"):
    filename.unlink()

print("Result file deletion done.")

for cve, sig_files in sig_file_dic.items():
    print(cve)
    if cve not in targets.index:
        continue
    target_bins = targets.loc[cve]['software'].to_list()
    
    # write sig file paths to file
    sig_list_file = os.path.join(OUTPUT_DIR,  cve+"_sig_list")
    with open(sig_list_file, "w") as f:
        for sig_path in sig_files:
            f.write(sig_path + " 1\n")
    
    for target_bin in target_bins:
        target_bin = convert_bin_path(target_bin)
        target_bin = target_bin.replace("O0", "O1")
        sym_file = target_bin+".sym"
        generate_sym(target_bin, sym_file)
        cmd = "python match_sig.py {} {} {}".format(target_bin, sym_file, sig_list_file)
        print(cmd)
        print(call(cmd, shell=True))
        
        # read the detection result

Result file deletion done.
CVE-2015-0293
python match_sig.py /extended/openssl/binary/O1/openssl-1.0.1n /extended/openssl/binary/O1/openssl-1.0.1n.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
python match_sig.py /extended/openssl/binary/O1/openssl-1.0.1l /extended/openssl/binary/O1/openssl-1.0.1l.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
python match_sig.py /extended/openssl/binary/O1/openssl-1.0.1a /extended/openssl/binary/O1/openssl-1.0.1a.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
CVE-2017-13010
python match_sig.py /extended/tcpdump/binary/O1/tcpdump-4.9.0 /extended/tcpdump/binary/O1/tcpdump-4.9.0.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2017-13010_sig_list
0
python match_sig.py /extended/tcpdump/binary/O1/tcpdump-4.9.3 /extended/tcpdump/binary/O1/tcpdump-4.9.3.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2017-13010_sig_list
0
CVE-2016-7122
python match_sig.py /extended/ffmpeg/

### Result Statistics

In [277]:
res_files = glob("match_res_*")
CVE_res = [] # (CVE, bin, P/N,Score)
Detection_times = []
for res_file in res_files:
    with open(res_file, "r") as f:
        lines = f.readlines()
        bin_name = res_file.split("_")[2]
        if len(lines)==1:
            continue
        CVE, IS_P, score = lines[0].strip().split(" ")
        CVE_res.append((CVE, bin_name, IS_P, score))

In [278]:
df_cve_res_O1 = pd.DataFrame(CVE_res, columns=["CVE", "software", "patched", "score"])

In [279]:
all_results_O1 = pd.merge(df_cve_res, targets_all, on=["CVE","software"], how="inner")

In [280]:
all_results_O1.shape

(167, 6)

### Accuracy in Support

In [281]:
TPs = all_results_O1.query("patched=='P' and label==1")
TNs = all_results_O1.query("patched=='N' and label==-1")

In [282]:
correct_cases = TNs.shape[0] + TPs.shape[0]
accuracy = correct_cases*1.0/all_results_O1.shape[0]
correct_cases,accuracy

(88, 0.5269461077844312)

## Detection for O2

In [252]:
# delete all match result files.
for filename in Path("./").glob("match_res_*"):
    filename.unlink()

print("Result file deletion done.")

for cve, sig_files in sig_file_dic.items():
    print(cve)
    if cve not in targets.index:
        continue
    target_bins = targets.loc[cve]['software'].to_list()
    
    # write sig file paths to file
    sig_list_file = os.path.join(OUTPUT_DIR,  cve+"_sig_list")
    with open(sig_list_file, "w") as f:
        for sig_path in sig_files:
            f.write(sig_path + " 1\n")
    
    for target_bin in target_bins:
        target_bin = convert_bin_path(target_bin)
        target_bin = target_bin.replace("O0", "O2")
        sym_file = target_bin+".sym"
        generate_sym(target_bin, sym_file)
        cmd = "python match_sig.py {} {} {}".format(target_bin, sym_file, sig_list_file)
        print(cmd)
        print(call(cmd, shell=True))
        
        # read the detection result

Result file deletion done.
CVE-2015-0293
python match_sig.py /extended/openssl/binary/O2/openssl-1.0.1n /extended/openssl/binary/O2/openssl-1.0.1n.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
python match_sig.py /extended/openssl/binary/O2/openssl-1.0.1l /extended/openssl/binary/O2/openssl-1.0.1l.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
python match_sig.py /extended/openssl/binary/O2/openssl-1.0.1a /extended/openssl/binary/O2/openssl-1.0.1a.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
CVE-2017-13010
python match_sig.py /extended/tcpdump/binary/O2/tcpdump-4.9.0 /extended/tcpdump/binary/O2/tcpdump-4.9.0.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2017-13010_sig_list
0
python match_sig.py /extended/tcpdump/binary/O2/tcpdump-4.9.3 /extended/tcpdump/binary/O2/tcpdump-4.9.3.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2017-13010_sig_list
0
CVE-2016-7122
python match_sig.py /extended/ffmpeg/

### Result Statistics

In [267]:
res_files = glob("match_res_*")
CVE_res = [] # (CVE, bin, P/N,Score)
Detection_times = []
for res_file in res_files:
    with open(res_file, "r") as f:
        lines = f.readlines()
        bin_name = res_file.split("_")[2]
        if len(lines)==1:
            continue
        CVE, IS_P, score = lines[0].strip().split(" ")
        CVE_res.append((CVE, bin_name, IS_P, score))

In [268]:
df_cve_res_O2 = pd.DataFrame(CVE_res, columns=["CVE", "software", "patched", "score"])

In [269]:
all_results_O2 = pd.merge(df_cve_res_O2, targets_all, on=["CVE","software"], how="inner")

In [270]:
all_results_O2.shape

(167, 6)

### Accuracy in Support

In [271]:
TPs = all_results_O2.query("patched=='P' and label==1")
TNs = all_results_O2.query("patched=='N' and label==-1")

In [272]:
correct_cases = TNs.shape[0] + TPs.shape[0]
accuracy = correct_cases*1.0/all_results_O2.shape[0]
correct_cases,accuracy

(88, 0.5269461077844312)

## Detection for O3

In [288]:
# delete all match result files.
for filename in Path("./").glob("match_res_*"):
    filename.unlink()

print("Result file deletion done.")

for cve, sig_files in sig_file_dic.items():
    print(cve)
    if cve not in targets.index:
        continue
    target_bins = targets.loc[cve]['software'].to_list()
    
    # write sig file paths to file
    sig_list_file = os.path.join(OUTPUT_DIR,  cve+"_sig_list")
    with open(sig_list_file, "w") as f:
        for sig_path in sig_files:
            f.write(sig_path + " 1\n")
    
    for target_bin in target_bins:
        target_bin = convert_bin_path(target_bin)
        target_bin = target_bin.replace("O0", "O3")
        sym_file = target_bin+".sym"
        generate_sym(target_bin, sym_file)
        cmd = "python match_sig.py {} {} {}".format(target_bin, sym_file, sig_list_file)
        print(cmd)
        print(call(cmd, shell=True))
        
        # read the detection result

Result file deletion done.
CVE-2015-0293
python match_sig.py /extended/openssl/binary/O3/openssl-1.0.1n /extended/openssl/binary/O3/openssl-1.0.1n.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
python match_sig.py /extended/openssl/binary/O3/openssl-1.0.1l /extended/openssl/binary/O3/openssl-1.0.1l.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
python match_sig.py /extended/openssl/binary/O3/openssl-1.0.1a /extended/openssl/binary/O3/openssl-1.0.1a.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2015-0293_sig_list
0
CVE-2017-13010
python match_sig.py /extended/tcpdump/binary/O3/tcpdump-4.9.0 /extended/tcpdump/binary/O3/tcpdump-4.9.0.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2017-13010_sig_list
0
python match_sig.py /extended/tcpdump/binary/O3/tcpdump-4.9.3 /extended/tcpdump/binary/O3/tcpdump-4.9.3.sym /home/ubuntu/fiber/software_dataset/outputs/CVE-2017-13010_sig_list
0
CVE-2016-7122
python match_sig.py /extended/ffmpeg/

### Result Statistics

In [289]:
res_files = glob("match_res_*")
CVE_res = [] # (CVE, bin, P/N,Score)
Detection_times = []
for res_file in res_files:
    with open(res_file, "r") as f:
        lines = f.readlines()
        bin_name = res_file.split("_")[2]
        if len(lines)==1:
            continue
        CVE, IS_P, score = lines[0].strip().split(" ")
        CVE_res.append((CVE, bin_name, IS_P, score))

In [290]:
df_cve_res_O3 = pd.DataFrame(CVE_res, columns=["CVE", "software", "patched", "score"])

In [292]:
all_results_O3 = pd.merge(df_cve_res_O3, targets_all, on=["CVE","software"], how="inner")

In [293]:
all_results_O3.shape

(152, 6)

### Accuracy in Support

In [294]:
TPs = all_results_O3.query("patched=='P' and label==1")
TNs = all_results_O3.query("patched=='N' and label==-1")

In [295]:
correct_cases = TNs.shape[0] + TPs.shape[0]
accuracy = correct_cases*1.0/all_results_O3.shape[0]
correct_cases,accuracy

(77, 0.506578947368421)

In [296]:
TNs

,CVE,software,patched,score,func,label
7,CVE-2015-0206,openssl-1.0.1h,N,1.66,dtls1_buffer_record,-1
23,CVE-2014-3568,openssl-1.0.1i,N,2.44,ssl23_get_client_method,-1
24,CVE-2016-7502,ffmpeg-3.0.2,N,1.36,cavs_idct8_add_c,-1
30,CVE-2016-2180,openssl-1.0.1t,N,0.27,TS_OBJ_print_bio,-1
48,CVE-2016-6671,ffmpeg-3.0.9,N,2.60,raw_decode,-1
49,CVE-2016-6671,ffmpeg-3.0.9,N,2.60,raw_decode,-1
50,CVE-2016-6671,ffmpeg-3.0.9,N,3.96,raw_decode,-1
51,CVE-2016-6671,ffmpeg-3.0.9,N,3.96,raw_decode,-1
55,CVE-2015-0287,openssl-1.0.1l,N,0.52,ASN1_item_ex_d2i,-1
57,CVE-2016-10190,ffmpeg-3.0.1,N,1.20,http_read_stream,-1


In [297]:
TPs

,CVE,software,patched,score,func,label
0,CVE-2017-14055,ffmpeg-3.3.6,P,58.21,mv_read_header,1
2,CVE-2017-14767,ffmpeg-3.3.6,P,1.32,sdp_parse_fmtp_config_h264,1
4,CVE-2017-14059,ffmpeg-3.3.5,P,20.73,cine_read_header,1
10,CVE-2017-14054,ffmpeg-3.2.9,P,7.36,ivr_read_header,1
11,CVE-2017-14767,ffmpeg-3.3.5,P,1.61,sdp_parse_fmtp_config_h264,1
14,CVE-2014-3569,openssl-1.0.1u,P,3.73,ssl23_get_client_hello,1
15,CVE-2017-16840,ffmpeg-3.4.2,P,0.54,ff_vc2enc_init_transforms,1
27,CVE-2015-0288,openssl-1.0.1n,P,0.70,X509_to_X509_REQ,1
28,CVE-2017-14058,ffmpeg-3.3.6,P,5.77,read_data,1
29,CVE-2016-7905,ffmpeg-3.1.5,P,14.85,read_gab2_sub,1
